In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml
import sys 
import os
import numpy as np

In [3]:
main_dir = '/ocean/projects/asc170022p/singla/ExplainingBBSmoothly'
mimic_file_name = os.path.join(main_dir, 'data', 'MIMIC_CXR_PA_AP_views_image_report_labels_concepts.csv')

In [4]:
df_mimic = pd.read_csv(mimic_file_name)
print(df_mimic.shape)
df_mimic.head(2)

(215518, 48)


,original_index,Reports,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,...,hilar_opacity,interstitial_marking,enlarged_pulmonary_arteries,chf,pleural fluid,blunt,hilar_engorgement,air_bronchogram,heart size,pleural effusion
0,0,No acute cardiopulmonary abnormality.,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,1,No acute cardiopulmonary process.,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


# Read Classifier

In [25]:
config_filename = os.path.join(main_dir, 'configs/Step_1_StanfordCheXpert_Classifier_256.yaml')
config = yaml.load(open(config_filename))
#for k in config.keys():
#    print(k, config[k])
categories = config['categories'].split(',')

# Choose a concept

In [8]:
concept_name = 'cardiac silhouette'
concept_dir = os.path.join(main_dir, config['log_dir'], config['name'],
                           'Classifier_Output_MIMIC_Concepts', 
                            concept_name)
try:
    os.makedirs(concept_dir)
except:
    pass

In [9]:
concept_dir

'/ocean/projects/asc170022p/singla/ExplainingBBSmoothly/output/classifier/StanfordCheXpert_256/Classifier_Output_MIMIC_Concepts/cardiac silhouette'

In [10]:
positive_set = df_mimic.loc[df_mimic[concept_name] == 1]
p = positive_set.shape[0] 
negative_set = df_mimic.loc[df_mimic[concept_name] == 0]
n = negative_set.shape[0]
negative_set = negative_set.sample(n = min(n,p * 5))
print(p, negative_set.shape[0])

(1602, 8010)


In [11]:
if False:
    positive_file_names = np.asarray(positive_set[config['output_csv_names_column']])
    negative_file_names = np.asarray(negative_set[config['output_csv_names_column']])
    np.save(os.path.join(concept_dir, 'positive_names.npy'),positive_file_names)
    np.save(os.path.join(concept_dir, 'negative_names.npy'),negative_file_names)
else:
    positive_file_names = np.load(os.path.join(concept_dir, 'positive_names.npy'),
                                 allow_pickle=True)
    negative_file_names = np.load(os.path.join(concept_dir, 'negative_names.npy'),
                                 allow_pickle=True)
print(positive_file_names.shape, negative_file_names.shape)

((1602,), (8010,))


# Save activations

Next step is to save the activations of these images, so that we can train a linear regressor on them.

To do this, modify the config above to include:

In [26]:
tmp = 'negative'
config['batch_size'] = 50
config['feature'] = True
config['names'] = os.path.join(concept_dir, tmp+'_names.npy')
config['output_folder_name'] = 'Classifier_Output_MIMIC_Concepts/'+concept_name
config['partition'] = concept_name + '_' + tmp
config_filename_1 =config_filename[:-5] + '_temp.yaml'
with open(config_filename_1, 'w') as outfile:
    yaml.dump(config, outfile, default_flow_style=False)

In [27]:
cmd = ['python', 'ExplainingBBSmoothly/Step_1_test_classifier.py', '-c', config_filename_1]
print(" ".join(cmd))

python ExplainingBBSmoothly/Step_1_test_classifier.py -c /ocean/projects/asc170022p/singla/ExplainingBBSmoothly/configs/Step_1_StanfordCheXpert_Classifier_256_temp.yaml


In [31]:
positive_feature = np.load(os.path.join(concept_dir, 'feature_dense_4_'+concept_name+'_positive.npy'))
negative_feature = np.load(os.path.join(concept_dir, 'feature_dense_4_'+concept_name+'_negative.npy'))
positive_feature.shape, negative_feature.shape

((1100, 8, 8, 1024), (5500, 8, 8, 1024))

In [32]:
# Max the filters and save as CSV to be used in R-code
positive_feature = np.max(positive_feature, axis=(1,2))
negative_feature = np.max(negative_feature, axis=(1,2))
positive_feature.shape, negative_feature.shape

((1100, 1024), (5500, 1024))

In [33]:
# save half of the data as test set
p = positive_feature.shape[0]
n = negative_feature.shape[0]
positive_feature_train = positive_feature[0:int(p/2),:]
positive_feature_test = positive_feature[int(p/2):,:]
negative_feature_train = negative_feature[0:int(n/2),:]
negative_feature_test = negative_feature[int(n/2):,:]
print(positive_feature_train.shape, negative_feature_train.shape)
print(positive_feature_test.shape, negative_feature_test.shape)

((550, 1024), (2750, 1024))
((550, 1024), (2750, 1024))


In [34]:
np.savetxt(os.path.join(concept_dir, 'feature_dense_4_'+concept_name+'_max_positive.csv'),
        positive_feature_train,
        delimiter=",")
np.savetxt(os.path.join(concept_dir, 'feature_dense_4_'+concept_name+'_max_negative.csv'),
        negative_feature_train,
        delimiter=",")

# Run lasso regression

In [22]:
file_name_suffix = 'feature_dense_4_'+concept_name+'_max'

In [23]:
R_file_name = os.path.join(main_dir, 'Step_5_Lasso_Regression.r')
slurm_file_name = os.path.join(main_dir, 'jobs', 'slurmLauncher_LM.sh')

In [24]:
cmd = ['/usr/bin/Rscript', '--vanilla',R_file_name,\
'--concept_dir',concept_dir, '--file_name', file_name_suffix,\
       '--measure', 'auc', ]
#s = subprocess.check_output(cmd)
print(" ".join(cmd))

/usr/bin/Rscript --vanilla /ocean/projects/asc170022p/singla/ExplainingBBSmoothly/Step_5_Lasso_Regression.r --concept_dir /ocean/projects/asc170022p/singla/ExplainingBBSmoothly/output/classifier/StanfordCheXpert_256/Classifier_Output_MIMIC_Concepts/cardiac silhouette --file_name feature_dense_4_cardiac silhouette_max --measure auc


# Read the output of lasso regression to get the concept vector 

## Helper functions

In [99]:
import re
def get_features_max(text, label = 'X'):
    """
    Get the non-zero Important Features - 1se Lambda 
    from the 10 lasso-classification runs 
    """
    text = text.replace("[1] \"(Intercept)\"", "")
    text = text.split("\"Important Features - 1se Lambda\"")[1:]
    text = [t.split('\n[1]')[0] for t in text]
    text[9] = text[9].split('\n\tStability ')[0]
    text = [t.replace("\n", "") for t in text]
    text = [t.replace("\"", "") for t in text]
    text = [re.sub("\[\d+\]", "", t) for t in text]
    text = [t.strip() for t in text]
    text = [t.split(label)[1:] for t in text]
    text = [[int(tt) for tt in t] for t in text]
    return text
def get_lasso_coeff_max(text):
    text = text.split("[1] \"Important Features Coeff - 1se Lambda\"")[1:]
    text = [t.split('[1] \"Important Features - Min Lambda\"')[0] for t in text]
    text = [t.replace("\n", "") for t in text]
    text = [t.replace("\"", "") for t in text]
    text = [t.replace("  ", " ") for t in text]
    text = [re.sub("\[\d+\]", "", t) for t in text]
    text = [t.strip() for t in text]
    text = [t.split(' ') for t in text]
    
    for i in range(0, len(text)):
        temp = []
        for j in range(len(text[i])):
            try:
                temp.append(float(text[i][j]))
            except:
                pass
        text[i] = temp
    return text
def get_filters(features):
    features = np.asarray(features)
    features = features - 1
    features = np.asarray(features)
    return(features)
def get_voting(features):
    n = len(features)
    feature_count = {}
    for i in range(0, n):
        for f in features[i]:
            if feature_count.has_key(f):
                feature_count[f]+=1
            else:
                feature_count[f] = 1
    sorted_dict = (sorted(feature_count.items(), key = 
             lambda kv:[kv[1], kv[0]]))
    sorted_dict.reverse()
    return sorted_dict
def choose_final_features(sorted_dict, n, thresh=0.5):
    consider_features = []
    for k in sorted_dict:
        if k[1] >= (n*thresh):
            consider_features.append(k)
    consider_features =  np.asarray(consider_features)
    return consider_features[:,0]

def which_lasso(final_features, features):
    num_common = []
    for f in features:
        l = set(final_features).intersection(set(f))
        num_common.append(len(f) - len(l))
    
    index = np.where(num_common == np.min(num_common))[0]
    max_f = 0
    for i in range(index.shape[0]):
        curr_index = index[i]
        if len(features[curr_index]) > max_f:
            max_f = len(features[curr_index])
            selected = curr_index       
    return selected

def y_pred_from_lasso(lasso_coeff, features,data):
    # f(x) = 1 + x_0 * beta_0 + ... + x_n-1 * beta_n-1
    value = np.ones(data.shape[0]) * lasso_coeff[0]  #intercept
    for i in range(len(features)):
        value += data[:,features[i]-1] * lasso_coeff[i+1]
    return value
def get_auc(lasso_coeff, features,data,label):
    from sklearn import metrics
    value = y_pred_from_lasso(lasso_coeff, features,data)
    fpr, tpr, thresholds = metrics.roc_curve(label, value, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    cm = metrics.confusion_matrix(label, (value>0).astype(int))
    recall = metrics.recall_score(label, (value>0).astype(int))
    return auc, cm , recall

## Get important units for a given concept

In [69]:
output_file = os.path.join(concept_dir, file_name_suffix+ '_output_R.txt')
f1 = open(output_file)
text = f1.read()
f1.close()

In [71]:
lasso_features = get_features_max(text)
lasso_features_coeff = get_lasso_coeff_max(text)
feature_count = get_voting(lasso_features)

In [72]:
# Key: unit number value: number of independent lasso runs in which this 
# unit was select as important
feature_count

[(1000, 10),
 (994, 10),
 (992, 10),
 (975, 10),
 (959, 10),
 (948, 10),
 (945, 10),
 (851, 10),
 (819, 10),
 (750, 10),
 (743, 10),
 (734, 10),
 (674, 10),
 (656, 10),
 (630, 10),
 (535, 10),
 (463, 10),
 (244, 10),
 (50, 10),
 (1, 10),
 (532, 9),
 (1003, 8),
 (958, 8),
 (636, 8),
 (296, 6),
 (202, 6),
 (152, 6),
 (971, 4),
 (767, 4),
 (342, 4),
 (320, 4),
 (235, 4),
 (126, 4),
 (957, 2),
 (911, 2),
 (904, 2),
 (883, 2),
 (826, 2),
 (787, 2),
 (665, 2),
 (658, 2),
 (608, 2),
 (468, 2),
 (418, 2),
 (380, 2),
 (331, 2),
 (225, 2),
 (117, 2),
 (42, 2)]

In [77]:
final_features = choose_final_features(feature_count, len(lasso_features), thresh=0.7)

In [96]:
final_features

array([1000,  994,  992,  975,  959,  948,  945,  851,  819,  750,  743,
        734,  674,  656,  630,  535,  463,  244,   50,    1,  532, 1003,
        958,  636])

In [81]:
lasso_choozen = which_lasso(final_features, lasso_features)
lasso_choozen

0

### Check the performance of this lasso regressor on the held out data

In [88]:
data = np.append(positive_feature_test,
                 negative_feature_test,
                 axis=0)
label = np.append(np.ones(positive_feature_test.shape[0]),\
                  np.zeros(negative_feature_test.shape[0]),axis=0)

In [89]:
data.shape, label.shape

((3300, 1024), (3300,))

In [90]:
auc,cm,recall = get_auc(lasso_features_coeff[lasso_choozen], 
                        lasso_features[lasso_choozen],
                        data,
                        label)
auc,cm,recall

(0.9886095867768595, array([[2740,   10],
        [  92,  458]]), 0.8327272727272728)

In [103]:
print("Concept units")
get_filters(lasso_features[lasso_choozen])

Concept units


array([   0,   49,  243,  462,  531,  534,  629,  635,  655,  673,  733,
        742,  749,  818,  850,  944,  947,  957,  958,  974,  991,  993,
        999, 1002])

In [102]:
print("Beta/magnitude of importance for concept units")
lasso_features_coeff[lasso_choozen]

Beta/magnitude of importance for concept units


[0.016468704,
 0.146722841,
 -0.792116561,
 -0.106157447,
 -0.204449296,
 -0.021153695,
 0.041747283,
 0.091853432,
 0.012680697,
 0.051987369,
 -0.06384932,
 0.120249635,
 0.086339685,
 -0.223706198,
 0.111918959,
 -0.57121582,
 0.006738822,
 0.308214648,
 0.000494669,
 0.04723585,
 0.09902543,
 0.081826857,
 -0.014118703,
 0.165001617,
 0.029055321]